In [115]:
import pandas as pd
import numpy as np
import os
import torch
import os
import sys
import json
import time
import requests
import datetime
import dateutil
from dateutil.relativedelta import relativedelta
import re
import xml.etree.ElementTree as et 
import tarfile
from pathlib import Path
from collections import defaultdict

In [158]:
nyt_api_key = '6iiiMFc1qUHG4P4GtEvsZwv5oaAvCbrn'
identity_terms = [
    'muslim',
    'jew',
    'jews',
    'white',
    'islam',
    'blacks',
    'muslims',
    'women',
    'whites',
    'gay',
    'black',
    'democrat',
    'islamic',
    'allah',
    'jewish',
    'lesbian',
    'transgender',
    'race',
    'brown',
    'woman',
    'mexican',
    'religion',
    'homosexual',
    'homosexuality',
    'africans'
]

In [8]:

start_date_str = "1987-01-01 00:00:00.0"
end_date_str = "2007-06-19 00:00:00.0"
# start = datetime.date.strftime(start_date_str, '%Y-%m-%d %H:%M:%S.%f')
# end = datetime.date.strftime(end_date_str, '%Y-%m-%d %H:%M:%S.%f')
# months_in_range = [x.split(' ') for x in pd.date_range(start, end, freq='MS').strftime("%Y %-m").tolist()]

api_key = "6iiiMFc1qUHG4P4GtEvsZwv5oaAvCbrn"

In [12]:
def untargzirator(fname, new_location):
    Path(new_location).mkdir(parents=True, exist_ok=True)
    if fname.endswith(".tgz"):
        tar = tarfile.open(fname, "r")
        tar.extractall(new_location)
        tar.close()
    elif fname.endswith("tar"):
        tar = tarfile.open(fname, "r:")
        tar.extractall(new_location)
        tar.close()

In [15]:
# uncomment to rextract
# extracted_files_path = '../data/nyt_corpus/extracted_data/'
# extracted_files_path_tar = '../data/nyt_corpus/tar_data/'
# year_files = os.listdir('../data/nyt_corpus/data')
# 
# for year in year_files:
#     months_zipped = os.listdir(f'../data/nyt_corpus/data/{year}/')    
#     new_location = extracted_files_path + year
#     for month in months_zipped:
#         untargzirator(f'../data/nyt_corpus/data/{year}/{month}', new_location)
#         print(new_location)            


../data/nyt_corpus/extracted_data/1987
../data/nyt_corpus/extracted_data/1987
../data/nyt_corpus/extracted_data/1987
../data/nyt_corpus/extracted_data/1987
../data/nyt_corpus/extracted_data/1987
../data/nyt_corpus/extracted_data/1987
../data/nyt_corpus/extracted_data/1987
../data/nyt_corpus/extracted_data/1987
../data/nyt_corpus/extracted_data/1987
../data/nyt_corpus/extracted_data/1987
../data/nyt_corpus/extracted_data/1987
../data/nyt_corpus/extracted_data/1987
../data/nyt_corpus/extracted_data/1988
../data/nyt_corpus/extracted_data/1988
../data/nyt_corpus/extracted_data/1988
../data/nyt_corpus/extracted_data/1988
../data/nyt_corpus/extracted_data/1988
../data/nyt_corpus/extracted_data/1988
../data/nyt_corpus/extracted_data/1988
../data/nyt_corpus/extracted_data/1988
../data/nyt_corpus/extracted_data/1988
../data/nyt_corpus/extracted_data/1988
../data/nyt_corpus/extracted_data/1988
../data/nyt_corpus/extracted_data/1988
../data/nyt_corpus/extracted_data/1989
../data/nyt_corpus/extrac

In [164]:
def contains_str(body, word) -> bool:
    if word in body and re.search(r"\b" + re.escape(word) + r"\b", body):
        return True
    else:
        return False

def check_if_article_contains_identifier(article_dict: dict, path: str) -> None:
    xtree = et.parse(path)
    xroot = xtree.getroot() 
    search = xroot.findall(".//body/body.content/*[@class='full_text']")
    full_text = "" 
    for node in search:
        for para in node:
            full_text += para.text
            full_text += '\n'
    
    full_text = full_text.lower()
    identifiers_found = list()
    for identifier in identity_terms:
        if contains_str(full_text, identifier):
            identifiers_found.append(1)
        else:
            identifiers_found.append(0)
    if sum(identifiers_found) > 0:
        identifiers_found.append(full_text)
        article_dict[path] = identifiers_found

In [163]:
# article_dict = dict()
# check_if_article_contains_identifier(article_dict, '../data/nyt_corpus/extracted_data/1996/03/02/0833741.xml')    
# article_dict

{'../data/nyt_corpus/extracted_data/1996/03/02/0833741.xml': [0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  "to the editor:\nre your feb. 26 news article on rural new york communities that are clamoring for new prisons:\nalthough african-americans and hispanic americans make up less than 25 percent of new york state's population, they represent 83 percent of the inmates confined in the state's prisons. seventy percent of those incarcerated in the state's correctional institutions come from new york city.\na distressing blend of politics and economics has developed. upstate, mainly  republican officials press for harsher sentencing laws and expensive prison construction not because such measures will prevent crime, but because they translate into a job development strategy for rural and predominantly white communities. low-income minority inmates provide the flesh-and-blood grist for this legislative pork-bar

In [165]:
col_identity_terms = identity_terms + ['full_text']
year_files = os.listdir('../data/nyt_corpus/extracted_data')
for year in year_files:
    months = os.listdir(f'../data/nyt_corpus/extracted_data/{year}/')
    for month in months:
        article_dict = dict()
        days = os.listdir(f'../data/nyt_corpus/extracted_data/{year}/{month}/')
        for day in days:
            files = os.listdir(f'../data/nyt_corpus/extracted_data/{year}/{month}/{day}/')
            for file in files:
                check_if_article_contains_identifier(article_dict, f'../data/nyt_corpus/extracted_data/{year}/{month}/{day}/{file}')
        month_df = pd.DataFrame.from_dict(article_dict, orient='index', columns=col_identity_terms)
        print(f'Saving {year}, {month} df, shape:', month_df.shape)
        month_df.to_csv(f'../data/nyt_corpus/filtered_data/identifier_articles_{year}_{month}.csv')

Saving 1987, 01 df, shape: (2705, 26)
Saving 1987, 02 df, shape: (2577, 26)
Saving 1987, 03 df, shape: (2938, 26)
Saving 1987, 04 df, shape: (2700, 26)
Saving 1987, 05 df, shape: (2975, 26)
Saving 1987, 06 df, shape: (2657, 26)
Saving 1987, 07 df, shape: (2715, 26)
Saving 1987, 08 df, shape: (2713, 26)
Saving 1987, 09 df, shape: (2639, 26)
Saving 1987, 10 df, shape: (2818, 26)
Saving 1987, 11 df, shape: (3013, 26)
Saving 1987, 12 df, shape: (2788, 26)
Saving 1988, 01 df, shape: (2748, 26)
Saving 1988, 02 df, shape: (2691, 26)
Saving 1988, 03 df, shape: (2809, 26)
Saving 1988, 04 df, shape: (2759, 26)
Saving 1988, 05 df, shape: (2783, 26)
Saving 1988, 06 df, shape: (2655, 26)
Saving 1988, 07 df, shape: (2648, 26)
Saving 1988, 08 df, shape: (2496, 26)
Saving 1988, 09 df, shape: (2816, 26)
Saving 1988, 10 df, shape: (2870, 26)
Saving 1988, 11 df, shape: (2695, 26)
Saving 1988, 12 df, shape: (2483, 26)
Saving 1989, 01 df, shape: (2635, 26)
Saving 1989, 02 df, shape: (2444, 26)
Saving 1989,

In [240]:
# All text is lower case https://stackoverflow.com/questions/4576077/how-can-i-split-a-text-into-sentences
# -*- coding: utf-8 -*-
import re
alphabets= "([A-Za-z])"
prefixes = "(mr|st|mrs|ms|dr)[.]"
suffixes = "(inc|ltd|jr|sr|co)"
starters = "(mr|mrs|ms|dr|he\s|she\s|it\s|they\s|their\s|our\s|we\s|but\s|however\s|that\s|this\s|wherever)"
acronyms = "([a-z][.][a-z][.](?:[a-z][.])?)"
websites = "[.](com|net|org|io|gov)"
digits = "([0-9])"

def split_into_sentences(text):
    text = " " + text + "  "
    text = text.replace("\n"," ")
    text = re.sub(digits + "[.]" + digits,"\\1<prd>\\2",text)
    text = re.sub(prefixes,"\\1<prd>",text)
    text = re.sub(websites,"<prd>\\1",text)
    if "Ph.D" in text: text = text.replace("ph.d.","ph<prd>d<prd>")
    text = re.sub("\s" + alphabets + "[.] "," \\1<prd> ",text)
    text = re.sub(acronyms+" "+starters,"\\1<stop> \\2",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>\\3<prd>",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>",text)
    text = re.sub(" "+suffixes+"[.] "+starters," \\1<stop> \\2",text)
    text = re.sub(" "+suffixes+"[.]"," \\1<prd>",text)
    text = re.sub(" " + alphabets + "[.]"," \\1<prd>",text)
    if "”" in text: text = text.replace(".”","”.")
    if "\"" in text: text = text.replace(".\"","\".")
    if "!" in text: text = text.replace("!\"","\"!")
    if "?" in text: text = text.replace("?\"","\"?")
    text = text.replace(".",".<stop>")
    text = text.replace("?","?<stop>")
    text = text.replace("!","!<stop>")
    text = text.replace("<prd>",".")
    sentences = text.split("<stop>")
    sentences = sentences[:-1]
    sentences = [s.strip() for s in sentences]
    return sentences



In [241]:
#,nyt_text,hate,keyword
def create_final_df(df, df_row):
    in_article = [col for col in identity_terms if row[col] == 1]
    article_sentences = split_into_sentences(df_row['full_text'])
    for sentence in article_sentences:
        for identifier in in_article:
            if identifier in sentence and re.search(r"\b" + re.escape(identifier) + r"\b", sentence):
                df = df.append({'nyt_text': sentence,'hate': 0,'keyword':identifier}, ignore_index=True)
    
    return df

In [243]:
nyt_all_df = pd.DataFrame(columns=['nyt_text','hate','keyword']) 
#%
for filtered_file in os.listdir('../data/nyt_corpus/filtered_data/'):
    print(filtered_file)
    temp_df = pd.read_csv(f'../data/nyt_corpus/filtered_data/{filtered_file}')
    for _, row in temp_df.iterrows():
        nyt_all_df = create_final_df(nyt_all_df, row)

identifier_articles_1987_01.csv
identifier_articles_1987_02.csv
identifier_articles_1987_03.csv
identifier_articles_1987_04.csv
identifier_articles_1987_05.csv
identifier_articles_1987_06.csv
identifier_articles_1987_07.csv
identifier_articles_1987_08.csv
identifier_articles_1987_09.csv
identifier_articles_1987_10.csv
identifier_articles_1987_11.csv
identifier_articles_1987_12.csv
identifier_articles_1988_01.csv
identifier_articles_1988_02.csv
identifier_articles_1988_03.csv
identifier_articles_1988_04.csv
identifier_articles_1988_05.csv
identifier_articles_1988_06.csv
identifier_articles_1988_07.csv
identifier_articles_1988_08.csv
identifier_articles_1988_09.csv
identifier_articles_1988_10.csv
identifier_articles_1988_11.csv
identifier_articles_1988_12.csv
identifier_articles_1989_01.csv
identifier_articles_1989_02.csv
identifier_articles_1989_03.csv
identifier_articles_1989_04.csv
identifier_articles_1989_05.csv
identifier_articles_1989_06.csv
identifier_articles_1989_07.csv
identifi

In [244]:
nyt_all_df


,nyt_text,hate,keyword
0,"'' there are tiles featuring hares, black bear...",0,black
1,"smaller, portable electric boot dryers, sold t...",0,women
2,q.our concrete basement walls are covered with...,0,white
3,this neutralizes the alkalies and washes off t...,0,white
4,lead: the 17th century was an important period...,0,women
...,...,...,...
2398035,ms. lupino plays a widow terrorized by a deran...,0,women
2398036,mr. walsh's other broadway credits were for mu...,0,black
2398037,an accommodation with israel is imperative -- ...,0,jewish
2398038,if a rodriguez ''yankeeography'' becomes a rea...,0,woman


In [245]:
nyt_all_df.to_csv('../data/nyt_corpus/final_data/final_df.csv')



In [ ]:

# https://catalog.ldc.upenn.edu/LDC2008T19
# https://developer.nytimes.com/faq
# from January 1, 1987 and June 19, 2007

In [254]:
counts_of_nyt_identifiers = nyt_all_df.groupby('keyword').agg('count').sort_values("hate", ascending=True)
print(counts_of_nyt_identifiers)

               nyt_text    hate
keyword                        
transgender         552     552
allah              1549    1549
africans           7331    7331
homosexuality      7338    7338
jew                8961    8961
homosexual         9682    9682
lesbian           11578   11578
islam             16543   16543
muslims           24612   24612
whites            26446   26446
religion          33267   33267
islamic           40005   40005
mexican           40373   40373
muslim            41520   41520
blacks            45594   45594
jews              49187   49187
gay               60560   60560
democrat          95299   95299
jewish            98453   98453
brown            155894  155894
race             167223  167223
woman            237502  237502
black            348654  348654
women            396852  396852
white            473065  473065


In [260]:
temp_df = nyt_all_df[nyt_all_df['keyword'] == 'transgender']
temp_df


,nyt_text,hate,keyword
740179,the march -- officially called the 1993 march ...,0,transgender
860817,"""stonewall was a rebellion of transgender peop...",0,transgender
912950,"""the sex of the stars"" is being marketed as a ...",0,transgender
925605,a leadership group of labor activists gathered...,0,transgender
978182,this year's committee has adopted the name san...,0,transgender
...,...,...,...
2392098,"several unit officers are liaisons to the gay,...",0,transgender
2393121,the center was presenting its ''out at the mov...,0,transgender
2396091,"if anything, some said, the problems are far w...",0,transgender
2397237,also on monday night: the new york stop on the...,0,transgender


In [271]:
# note that the minimum sentences with an identifier is 'transgender' with 552 instances
uniform_sample_nyt =  pd.DataFrame(columns=['nyt_text','hate','keyword']) 
min_n = counts_of_nyt_identifiers.iloc[0,0]
for identifier in identity_terms:
    temp_df = nyt_all_df[nyt_all_df['keyword'] == identifier]
    sample_df = temp_df.sample(n=min_n, random_state=42, replace=False)
    uniform_sample_nyt = uniform_sample_nyt.append(sample_df)
    

In [272]:
uniform_sample_nyt = uniform_sample_nyt.sample(frac=1, random_state=42)
uniform_sample_nyt

,nyt_text,hate,keyword
2116647,"sopes, a mexican ''pizza,'' is a soft corn pan...",0,mexican
1043612,"for many black south africans, the prospects o...",0,africans
449364,they sought a safe haven where they could cont...,0,jewish
2322717,such black-and-white rules were well suited to...,0,black
1242853,if arabs and jews can peacefully co-exist in j...,0,jews
...,...,...,...
255545,there have been a series of anonymous hate let...,0,gay
1710562,embittered by his experience in the french arm...,0,africans
2091765,"in massachusetts, every state legislator on th...",0,gay
381244,"when jordan conquered east jerusalem in 1948, ...",0,jew


In [273]:
uniform_sample_nyt.to_csv('../data/nyt_corpus/final_data/final_df_uniform_sample.csv')